<a href="https://colab.research.google.com/github/JMQuinlan/Boundary-Layer/blob/main/Blasius_shooting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Integrate (solve) the Blasius Equation with the shooting method

copyright 2021 - J. M. Quinlan  
University of Colorado  
Colorado Springs  

In this notebook we integrate the Blasius Equation for boundary layer flow over a flat plat treating the BVP as a IVP.

In [1]:
import numpy as np
#import cmath
#import math
from scipy import integrate
#from scipy import linalg
#import scipy.fftpack
from matplotlib.pylab import plt
import matplotlib as mpl

In [2]:
# Order of problem
mx = 3

# Define system parameters
u = np.zeros((mx,1),dtype=complex) # Initial values for generalized coordinates
u[0] = 0.
u[1] = 0.
u[2] = 0.3320594


# Plotting stuff
color1 = color=(0,0,0.6)
color2 = color=(1,0,0)
color3 = color=(0.2,1,0.2)
fnmul = 2
figdim = (3.25*fnmul,2.5*fnmul)
# mpl.rc('text', usetex = True) # set if you want to use TeX for plot text
mpl.rc('text', usetex = False) # Faster than TeX and only need extra 'r' as in: r'\alpha'

In [3]:
# define system to be solved: du/dt = f(u,t)
def sysde(t, u):
    """
    Simple spring-mass-damper system - 2 DOF

    f     = u[0] = Nondim. streamfunction
    f'    = u[1] = Nondim. x-direction velocity
    f''   = u[2] = Related to velocity gradient  
    f'''  = u[3] = Related ot velocity curvature (comes from viscous term)

    Returns 
    du/dt=[    u[1]          ]
          [    u[2]          ]
          [-1./2.*u[0] * u[2]]
    """

    # Assign some variables for convenience of notation
    #x1 = u[0]
    #x2 = u[1]
    #v1 = u[2]
    #v2 = u[3]

    # Output from ODE function must be a COLUMN vector, with n rows
    n = len(u)
    dudt = np.zeros((n,1),dtype=complex)
    dudt[0] = u[1]
    dudt[1] = u[2]
    dudt[2] = -1./2.*u[0]*u[2]
    
    return dudt

In [4]:
# Start by specifying the integrator:
# use ``vode`` with "backward differentiation formula"
r = integrate.ode(sysde).set_integrator('zvode', method='bdf')

# Set the time range
t_start = 0.0
t_final = 25.0
delta_t = 0.01
# Number of time steps: 1 extra for initial condition
num_steps = int(np.floor((t_final - t_start)/delta_t) + 1)

# Set initial condition(s): for integrating variable and time
#x1_t_zero = 0.5
#T_t_zero = 295.0
r.set_initial_value(u, t_start)
#r.set_initial_value([CA_t_zero, T_t_zero], t_start)

# Additional Python step: create vectors to store trajectories
t = np.zeros((num_steps, 1))
f0 = np.zeros((num_steps, 1),dtype=complex)
f1 = np.zeros((num_steps, 1),dtype=complex)
f2 = np.zeros((num_steps, 1),dtype=complex)
data = np.zeros((mx,num_steps)) # Initial values for generalized coordinates
t[0] = t_start
f0[0] = u[0]
f1[0] = u[1]
f2[0] = u[2]
data[:,0] = u.real.T

# Integrate the ODE(s) across each delta_t timestep
kk = 1
while r.successful() and kk < num_steps:
    r.integrate(r.t + delta_t)

    # Store the results to plot later
    t[kk] = r.t
    f0[kk] = r.y[0]
    f1[kk] = r.y[1]
    f2[kk] = r.y[2]
    data[:,kk] = r.y.real.T
    kk += 1

# Plot the trajectories in two separate plots:
fig = plt.figure(22)
ax1 = plt.subplot(111)
# ax1.plot(t, f0.real, 'b')
ax1.plot(t, f1.real, 'r')
ax1.plot(t, f2.real, 'g')
ax1.set_xlim(t_start, 6.)
ax1.set_xlabel(r'$\eta$')
ax1.set_ylabel(" f', f'' ")
ax1.grid('on')

# ax2 = plt.subplot(212)
# ax2.plot(t, v1.real, 'b')
# ax2.plot(t, v2.real, 'r')
# ax2.set_xlim(t_start, t_final)
# ax2.set_xlabel('Time [s]')
# ax2.set_ylabel('Velocity [m]')
# ax2.grid('on')

# Save figure
fig.savefig('testfig.eps')
fig.savefig('testfig.pdf')
# Show figure
#plt.show()
plt.close()

In [5]:
print(f1[-1])

[1.00000001+0.j]


In [6]:
print(f1[491])
print(t[491])

[0.99000024+0.j]
[4.91]
